Importing section

In [98]:
import os
import string
from nltk import tokenize
from gensim.models import Word2Vec
from nltk import word_tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd

Reading the data from the folder and cleaning it.

In [99]:
sentences = []
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           s = s.lower()
           s = s.translate(str.maketrans('', '', string.punctuation))
           sentences.append(s)

tokensSentenceslist = []

for s  in sentences:
    wordsList = word_tokenize(s)
    tokensSentenceslist.append(wordsList)


##################### Uncomment below section for testing #########################
# print(len(sentences))
#
# for s in sentences:
#      print("The sentence is : ")
#      print(s)
#      print("-----------------------End of the sentence -------------")
#
# print (sentences)


# print (len(tokensSentenceslist))
# print (tokensSentenceslist)


In [100]:
print (len(sentences))
print(sentences[0:5])

38715
['advertising in television is becoming more and more clever', 'lots of those ads invade our subconsciousness and try to make us buy the product even if we do not actually need it', 'when it comes to children lots of them dont have enough money to afford these products', 'they dont even understand the concept of money', 'even if they wanted it they can not get the thing they were told they need']


Generating the Word2Vec Model

In [101]:
#model = Word2Vec(tokensSentenceslist, min_count=1)

#model = Word2Vec(tokensSentenceslist, vector_size=50, min_count=1, sg=1)
#model = Word2Vec(sentences=tokensSentenceslist, vector_size=100, workers=1, seed=42)

model = Word2Vec(window=10, min_count=2,workers=6,vector_size=50,seed=42,sg=0)
model.build_vocab(tokensSentenceslist, progress_per=1000)
model.train(tokensSentenceslist, total_examples=model.corpus_count, epochs=model.epochs)


##################### Uncomment below section for testing #########################


# print(list(model.wv.index_to_key))
# print(len(list(model.wv.index_to_key)))

(2373525, 3636460)

In [63]:
model.corpus_count
#model.epochs

38715

In [ ]:
model.wv.most_similar("television")
model.wv.most_similar("argument")
model.wv.similarity("tv","against")

In [105]:
def vectorize(list_of_docs, model, strategy):
    """Generate vectors for list of documents using a Word Emx`bedding.

    Args:
        list_of_docs: List of documents.
        model: Gensim Word Embedding.
        strategy: Aggregation strategy ("average", or "min-max".)

    Raises:
        ValueError: If the strategy is other than "average" or "min-max".

    Returns:
        List of vectors.
    """
    features = []
    size_output = model.vector_size
    embedding_dict = model

    if strategy == "min-max":
        size_output *= 2

    if hasattr(model, "wv"):
        embedding_dict = model.wv

    for tokens in list_of_docs:
        zero_vector = np.zeros(size_output)
        vectors = []
        for token in tokens:
            if token in embedding_dict:
                try:
                    vectors.append(embedding_dict[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            if strategy == "min-max":
                min_vec = vectors.min(axis=0)
                max_vec = vectors.max(axis=0)
                features.append(np.concatenate((min_vec, max_vec)))
            elif strategy == "average":
                avg_vec = vectors.mean(axis=0)
                features.append(avg_vec)
            else:
                raise ValueError(f"Aggregation strategy {strategy} does not exist!")
        else:
            features.append(zero_vector)
    return features

-0.1692446

In [106]:
vectorized_docs = vectorize(tokensSentenceslist, model=model, strategy="average")



In [162]:
len(vectorized_docs), len(vectorized_docs[0])
print(model.wv["argument"])

print("#######################################################")
print(vectorized_docs[0])

[ 0.40850586 -0.34978685 -1.959126    2.6344159   0.8636176   0.73862237
 -0.01656396  2.1689315   0.09455873  2.232262    1.0606693   0.6851661
  1.2025019  -0.62458533 -0.8933515   0.15206935  0.7622899   0.23542207
  1.1568854  -0.5191893   1.1853704   0.35496783  1.3046758  -0.6329307
  0.23321836 -0.18543202  1.7009146  -0.8653078   0.6799102   3.2721758
 -0.23107289  1.178943   -0.13785091 -2.7942522  -1.4669671  -1.5336914
 -0.1132482  -3.6796699   0.6580933   0.55389977  0.06629828 -0.5170109
  1.0349716  -0.06024138 -1.1638317  -1.7385219   0.52351743  0.2860309
 -0.41214272 -0.5447228 ]
#######################################################
[ 0.41131788 -0.11716345  0.28971842  0.5577258  -0.5772017   0.9901962
  0.96931005 -0.5613676  -0.3226952  -0.4045963   0.18726283  0.44057044
 -0.42001647 -0.4549887  -0.8095281   0.07320678 -0.20656103 -0.8354379
  0.20628421 -0.6952043  -0.29465434 -0.11770947 -0.03002735  0.10196522
  0.37290215 -0.10642844  0.10388153 -0.5029228  -

In [89]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_


[ 1.0834987   1.0775832  -2.4190793   1.3539352   0.7693121  -0.01766455
 -0.53748876  2.1623745  -0.5207145   1.694928   -0.2794287   1.4977117
  0.3757558  -1.0392004  -1.1415775  -0.37531468  0.23239893  1.364053
  1.6795251   0.6714833   0.8824193   1.365521    1.8032991  -0.67293257
  1.0104337  -0.03604451  1.0017824  -0.5705173  -0.49769476  1.6430551
 -1.352539    1.2772088   0.03308525 -2.0387952  -2.4316406  -1.2589905
  0.95908076 -2.2138608   0.6789476  -1.0684443   0.03440151  0.9796872
  0.9881222  -1.3170159  -2.6712847  -1.2676587   1.5911027  -0.6523292
 -1.2096913   0.46780825]
#######################################################
[ 0.09301123  0.02892222  0.02476533  0.29560983 -0.7727718   0.23872662
  0.4515218  -0.06879666 -0.03741706 -0.3008877  -0.20542604  0.8528051
 -0.14720449 -0.2436079  -1.0440915  -0.17934558 -0.21951494 -0.6070896
  0.41599515  0.6486483  -0.45739534  0.38212645  0.3662845   0.4625144
  0.98150975 -0.35466102  0.38813674 -0.44521755 -1.

In [108]:
clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=10, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "tokens": [" ".join(text) for text in tokensSentenceslist],
    "cluster": cluster_labels
})

### Evaluate top terms of the cluster

In [196]:
print("Top terms per cluster (based on centroids):")
for i in range(10): # number of cluster k should be put here!!
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=3)
    print(clustering.cluster_centers_[i])
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    #print(f"Cluster {i}: {tokens_per_cluster}")

Top terms per cluster (based on centroids):
[ 1.19385659 -0.82888172  1.58800147  0.10622866 -0.04769936  0.53634079
  1.2926703  -0.58823001  0.37684249 -0.0375403   1.1391392  -0.21388091
 -0.8419543  -0.18153256  0.01115879  0.0577653  -0.66395227  0.1070669
  0.3855902   0.07921372  0.25195381 -0.24987248  0.91514523  0.87854672
 -0.2781197  -1.14427267  0.05927777  0.0140876  -1.56937785  0.24000135
  0.75462581 -0.68548434 -0.51476748  0.84428861 -0.97498851  0.26553429
  0.41518204 -0.35250388 -1.04682077  0.19067908 -0.42692063  0.4745817
  0.7416655  -0.27141488  0.15662372 -0.24953608  0.13406592  0.3687836
 -0.95627766  0.222207  ]
[ 0.70034438 -0.29410066  0.99018799  0.20499822 -0.04243745  0.39972958
  0.64275578 -0.33103261 -0.0416056  -0.13465635  0.72499667  0.00946553
 -0.46231743  0.12923565 -0.10479293 -0.02107713 -0.41081607 -0.20373062
  0.31501559 -0.29613619  0.14241664 -0.22131774  0.29306809  0.58699511
  0.02967619 -0.48050582  0.10401114  0.04557529 -1.17843

### Retrieve a random sample of documents for a given cluster


In [208]:
for i,t in enumerate(df_clusters.query(f"cluster == {0}").sample(10).iterrows()):
    print(t[1]["text"])
    print("-------------")

so the children see the advertise and then the parents have two possibilities
-------------
they learn this important things indirect so they donâ´t realise it
-------------
no wonder that kids are being spoilt nowadays because their parents want them to have a good childhood and buy their children everything they wish for
-------------
when the children see those ads they make their parents buy them these toys and are willing to watch even more tv
-------------
if they then get the plane and realise that the plane cant fly they will be very disappointed
-------------
already they do not understand that their parents work for buying the plays and that the plays cost money
-------------
the kids will see something that they like on television and they will say to their parents that they like to have this thing for christmas
-------------
adverts aret usefull for young children because they cant even buy things by themself
-------------
also it could be a good way to teach the kids that 

In [151]:
df_clusters.shape

# df_clusters.tokens[0]
# df_clusters.head()

(38715, 3)

### Most representative clusters

In [224]:
test_cluster = 5
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
# print(most_representative_docs[0])
for d in most_representative_docs[:10]:
    print(sentences[d])
    print("-------------")

in my opinion the supporters of the ban have a point but they focus their concerns on the wrong medium since tvs have lost in importance as seen in one south park episode where cartman becomes a internet star and there is a christmas special to do something and one of the kids stars a petition to save the televison since thats what brought families together the best
-------------
so the question that posses itself is if it is okay to have global corporations subsequently implanting our desires in our brain and where do you draw the line between advertisments and brain control if you take coca cola for example a lot of studies have been made to show that it can cause serious health issues when consumed every day as well as when you consume too much of it for example bad teeth because of the sugar and other troubles you get for example stomach ulcers due to all the acids it contents
-------------
reproduction has been essential for the survival of every living being and by adapting and a

In [203]:
#array = clustering.cluster_centers_[0]
print(len(vectorized_docs))
#print(array)
array = vectorized_docs[120].reshape(1,-1)
convertedArray = array.astype(float)

clustering.predict(convertedArray)

38715


array([1])

In [191]:
    #print(array.reshape(1,-1))

print(array)

print(convertedArray)

[[ 0.41131788 -0.11716345  0.28971842  0.5577258  -0.5772017   0.9901962
   0.96931005 -0.5613676  -0.3226952  -0.4045963   0.18726283  0.44057044
  -0.42001647 -0.4549887  -0.8095281   0.07320678 -0.20656103 -0.8354379
   0.20628421 -0.6952043  -0.29465434 -0.11770947 -0.03002735  0.10196522
   0.37290215 -0.10642844  0.10388153 -0.5029228  -0.48719856  0.58507514
  -0.10985485 -0.43433896 -0.43355358 -0.5003478  -0.255086    1.0371667
  -0.56815755 -1.0393796  -0.58956033  0.4535848  -0.05363839 -0.7777182
   1.1643625  -0.07477699 -0.07372791 -0.9933421  -0.22898763 -0.10050163
  -0.13402921 -0.31821993]]
[[ 0.41131788 -0.11716345  0.28971842  0.55772579 -0.57720172  0.99019623
   0.96931005 -0.56136757 -0.3226952  -0.4045963   0.18726283  0.44057044
  -0.42001647 -0.45498869 -0.80952811  0.07320678 -0.20656103 -0.83543789
   0.20628421 -0.69520432 -0.29465434 -0.11770947 -0.03002735  0.10196522
   0.37290215 -0.10642844  0.10388153 -0.50292277 -0.48719856  0.58507514
  -0.10985485 

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_25384\3934604238.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  convertedArray = array.astype(np.float)


In [226]:
## testing
def vectorizeSentenceTest(sentences):
    tokensSentenceslist = []
    for s  in sentences:
        wordsList = word_tokenize(s)
        tokensSentenceslist.append(wordsList)
    return tokensSentenceslist

testTokens = vectorizeSentenceTest(["In my opinion, supporters of the ban have a position, and they are looking at the wrong medium, as seen in a South Park episode where Cartman becomes an internet sensation and there is a Christmas special to do something, and one of the kids stars a petition to save the tv news because it is the best way to bring families together."])

vectorized_docs_tesing = vectorize(testTokens, model=model, strategy="average")

def predictTest(vectorizedDocsTest):
    array = vectorizedDocsTest
    print(clustering.predict(array))
    return
#len(vectorized_docs_tesing), len(vectorized_docs_tesing[0])
#print(vectorized_docs_tesing)

predictTest(vectorized_docs_tesing)

[5]
